In [ ]:
import pandas as pd
import time
import random
from queue import PriorityQueue, Queue

# Load the dataset with error handling for column names
def load_games(file_path):
    try:
        df = pd.read_csv(file_path)
        # Try to find matching columns (case-insensitive)
        column_mapping = {
            'game': 'GAME',
            'ratio': 'RATIO',
            'gamers': 'GAMERS',
            'comp %': 'COMP %',
            'time': 'TIME',
            'rating': 'RATING',
            'added': 'ADDED',
            'true_achitgame_score': 'True_AchitGame_Score'
        }

        # Find actual column names in the dataframe
        actual_columns = {col.lower(): col for col in df.columns}

        # Map our desired columns to actual columns
        selected_columns = []
        for desired_col, display_name in column_mapping.items():
            if desired_col in actual_columns:
                selected_columns.append(actual_columns[desired_col])
            else:
                print(f"Warning: Column '{display_name}' not found in CSV file")

        games = df[selected_columns].dropna()
        return games.to_dict(orient='records')
    except Exception as e:
        print(f"Error loading games: {e}")
        return []

# Player class for the queue
class Player:
    def __init__(self, username, priority):
        self.username = username
        self.priority = priority  # 0 = VIP, 1 = Regular
        self.join_time = time.time()

    def __lt__(self, other):
        return self.priority < other.priority  # VIPs first

# Game Manager to simulate servers and queues
class CloudGamingManager:
    def __init__(self, game_list, server_limit=3):
        self.games = game_list
        self.vip_queue = PriorityQueue()
        self.regular_queue = Queue()
        self.playing_users = {}
        self.server_limit = server_limit

    def add_player(self, username, is_vip):
        player = Player(username, 0 if is_vip else 1)
        if is_vip:
            self.vip_queue.put(player)
            print(f"[+] VIP Player '{username}' joined the queue.")
        else:
            self.regular_queue.put(player)
            print(f"[+] Regular Player '{username}' joined the queue.")

    def assign_servers(self):
        while len(self.playing_users) < self.server_limit:
            next_player = None
            if not self.vip_queue.empty():
                next_player = self.vip_queue.get()
            elif not self.regular_queue.empty():
                next_player = self.regular_queue.get()
            else:
                break

            if next_player:
                game = random.choice(self.games)
                # Safely get game details with fallbacks
                game_title = game.get('GAME', game.get('game', 'Unknown Game'))
                game_rating = game.get('RATING', game.get('rating', 'N/A'))
                self.playing_users[next_player.username] = {
                    "start_time": time.time(),
                    "game": game
                }
                print(f"[🎮] '{next_player.username}' started playing '{game_title}' (Rating: {game_rating})")

    def release_players(self, play_duration=10):
        finished_users = []
        for username, session in self.playing_users.items():
            if time.time() - session['start_time'] > play_duration:
                finished_users.append(username)

        for user in finished_users:
            game_info = self.playing_users[user]['game']
            # Safely get completion and score with fallbacks
            completion = game_info.get('COMP %', game_info.get('comp %', 'N/A'))
            score = game_info.get('True_AchitGame_Score', game_info.get('true_achitgame_score', 'N/A'))
            game_title = game_info.get('GAME', game_info.get('game', 'Unknown Game'))
            print(f"[⏹️] '{user}' has finished playing '{game_title}' (Completion: {completion}%, Score: {score})")
            del self.playing_users[user]

    def show_status(self):
        print("\n--- Queue Status ---")
        print(f"VIP Queue: {self.vip_queue.qsize()}")
        print(f"Regular Queue: {self.regular_queue.qsize()}")
        print(f"Currently Playing: {list(self.playing_users.keys())}\n")

# Simulation
def simulate_cloud_gaming():
    games = load_games('/content/Gamepass_Games_v1.csv')
    if not games:
        print("Error: No games loaded. Please check your CSV file and column names.")
        return

    manager = CloudGamingManager(games)

    # Add test players
    players = [
        ("Nandhu", False),
        ("Ajay", True),
        ("Girish K T", False),
        ("Mutthu", True),
        ("Abeed", True),
        ("Uday Kumar", False),
    ]

    for name, vip in players:
        manager.add_player(name, vip)

    # Simulate multiple rounds
    for round in range(5):
        print(f"\n===== Round {round + 1} =====")
        manager.release_players(play_duration=5)
        manager.assign_servers()
        manager.show_status()
        time.sleep(2)  # simulate delay

if __name__ == "__main__":
    simulate_cloud_gaming()